<a href="https://colab.research.google.com/github/Alfred78w/AI_project/blob/main/tutorials/mistral_finetune_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [2]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 472, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 472 (delta 211), reused 159 (delta 159), pack-reused 223 (from 2)
Receiving objects: 100% (472/472), 243.32 KiB | 992.00 KiB/s, done.
Resolving deltas: 100% (251/251), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install all required dependencies:

In [3]:
!pip install -r /content/mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
 

## Model download

In [ ]:
!pip install huggingface_hub

In [ ]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/root/mistral_models/7B-v0.3'

In [ ]:
# Alternatively, you can download the model from mistral

# !wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2024-05-24 18:50:25--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.6.117, 104.26.7.117, 172.67.70.68, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.6.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  40.5MB/s    in 6m 3s   

2024-05-24 18:56:29 (38.1 MB/s) - ‘mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [ ]:
# !DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [ ]:
!ls /content/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [ ]:
!pip install pdfplumber
!pip install datasets

In [53]:
!pip install jsonlines

In [56]:
import os
import pdfplumber

def extract_text_from_pdfs(pdf_folder):
    texts = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    texts.append(page.extract_text())
    return "\n".join(texts)

pdf_folder = "/content/drive/MyDrive/CV"
data_text = extract_text_from_pdfs(pdf_folder)
with open("extracted_text.txt", "w") as f:
    f.write(data_text)


"""# Prepare data and split (paragraph )"""
encoding = 'windows-1252'
# Load the text file
with open("extracted_text.txt", "r", encoding=encoding) as file:
    data = file.read()  # Read the entire text

# Split data into paragraphs or sections based on double newlines
paragraphs = data.split("\n")  # Assuming double newlines separate paragraphs

# Clean the data (optional)
paragraphs = [para.strip() for para in paragraphs if para.strip()]
print(f"Number of paragraphs: {len(paragraphs)}")

Number of paragraphs: 120


In [57]:
import pandas as pd
from datasets import Dataset

# Assuming 'paragraphs' is your list of strings
df = pd.DataFrame(paragraphs, columns=['Text'])

In [58]:
print(df)

                                                  Text
0                                   Alpha Mohamed KABA
1                    alpha.kaba@centrale-casablanca.ma
2                          https://kamweb.ga/portfolio
3       Bouskoura ville verte,27182 +212 (0) 658891986
4                                           FORMATIONS
..                                                 ...
115  Google Project management SIANA & ECC | Projet...
116  ESSEC Business School Septembre 2022 - FÃ©vrie...
117  L'excellence opÃ©rationnelle en Identification...
118  pratique Etude des systÃ¨mes de suivi basÃ©s s...
119  Learn Quest Scrum master Base de donnÃ©es rela...

[120 rows x 1 columns]


In [59]:
%cd /content/

/content


In [60]:
# make a new directory called data
!mkdir -p data

In [61]:
# navigate to this data directory
%cd /content/data

/content/data


In [47]:
# read data into a pandas dataframe
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [48]:
df.head()

,prompt,prompt_id,messages
0,"This story begins with an end. In March 1991, ...",5ee2fbb48ef35593b81444d7aec405bb4f152abbe80f7b...,[{'content': 'This story begins with an end. I...
1,Explain how the invention and widespread use o...,fc6aae406cd26c79db4d35dd32bcbd8ee0f1493a0096b5...,[{'content': 'Explain how the invention and wi...
2,Read the passage below and answer the question...,44a13514d9cd363d85479ff25e5837c60c5f90815428c2...,[{'content': 'Read the passage below and answe...
3,Explain the influence of culture on attitudes ...,c0c7f2a08bd4dc84bc527d774b1fe411eefa7bcdb847b5...,[{'content': 'Explain the influence of culture...
4,Can you provide data on the employment rates i...,b26cb026578e891c3ccd0cf075da6cffaa05df05412aa0...,[{'content': 'Can you provide data on the empl...


In [ ]:
df2['messages'][0]

In [ ]:
df2['prompt'][0]

In [62]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [63]:
df_train

,Text
82,MSI consulting | Stagiaire Assistant-IngÃ©nieur
94,"Data Analysis, apprentissage Juillet 2022 - Ao..."
53,COMPÃ‰TENCES
74,"FÃ©vrier 2024 | UM6P, Maroc"
69,Option : Sciences de donnÃ©es et digitalisatio...
...,...
91,StratÃ©gie de communication de la marque
14,2019
89,DÃ©composition des prix de vente du produit
79,"OpenAI API, Mistral, llama index"


In [64]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [65]:
!ls /content/data

ultrachat_chunk_eval.jsonl  ultrachat_chunk_train.jsonl


In [66]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [67]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that don't have the right format:

!python -m utils.reformat_data /content/data/ultrachat_chunk_train.jsonl

In [68]:
# eval data looks all good
!python -m utils.reformat_data /content/data/ultrachat_chunk_eval.jsonl

In [69]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/content/mistral-finetune/utils/validate_data.py", line 372, in <module>
    main(args)
  File "/content/mistral-finetune/utils/validate_data.py", line 160, in main
    train_args = TrainArgs.load(args.train_yaml)
  File "/usr/local/lib/python3.10/dist-packages/simple_parsing/helpers/serialization/serializable.py", line 309, in load
    return load(cls, path=path, drop_extra_fields=drop_extra_fields, load_fn=load_fn, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/simple_parsing/helpers/serialization/serializable.py", line 543, in load
    return from_dict(cls, d, drop_extra_fields=drop_extra_fields)
  File "/usr/local/lib/python3.10/dist-packages/simple_parsing/helpers/serialization/serializable.py", line 847, in from_dict
    f

## Start training

In [19]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [ ]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [ ]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml

2024-05-24 18:58:16.690967: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-24 18:58:17.292359: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 18:58:17.292438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 18:58:17.418671: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 18:58:17.481373: I tensorflow/core/platform/cpu_feature_guar

## Inference

In [ ]:
!pip install mistral_inference

In [ ]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

Machine learning is a subset of artificial intelligence that involves the use of algorithms to learn from data and make predictions or decisions without being explicitly programmed. It is a type of computer science that enables machines to learn and improve from experience without being explicitly programmed. Machine learning algorithms can learn from data and make predictions or decisions based
